In [1]:
##################################
# Calculate HGM
##################################

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import rasterio
import datetime
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from geopandas.io.arrow import _arrow_to_geopandas
from rasterio.mask import mask
from rasterio.plot import show
from shapely.geometry import shape, box, MultiLineString, LineString, Point, Polygon, MultiPolygon
from shapely import wkb, wkt
from functools import reduce
from pathlib import Path
from fsspec.implementations.http import HTTPFileSystem

ac_ = 0.000247105
ha_ = 0.0001

In [3]:

######## EDIT THESE ########
savefile = 'finaltests'
workspace = r'D:\gis\projects\YazooHGM\finalTracts'

foiin = os.path.join(workspace, 'halloween_site_locations_names.shp')
uniqueIDName = 'SiteName'
######## STOP EDIT ########


In [4]:
# cogs for flooding
floodar = ['https://giscog.blob.core.windows.net/publiccogs/alt2_2year_cog.tif','https://giscog.blob.core.windows.net/publiccogs/alt2_5year_cog.tif']
# Features of interest that HGM will be calculated within

# calculates unique id and original foi acres
foi = gpd.read_file(foiin).to_crs(5070)
#foi = foi.dissolve()
foi['uid'] = foi.index.map(hash)
foi['origacres'] = foi.area * ac_

# Clipping all data to the area of interest (aoi)
aoiurl = 'https://giscog.blob.core.windows.net/publicparquet/YazooAOI.parquet'
aoi = gpd.GeoDataFrame(pd.read_parquet(aoiurl))
aoi['geometry'] = aoi['geometry'].apply(wkb.loads)
aoi = aoi.set_geometry('geometry')
aoi = aoi.set_crs(5070)
# Features of interest that HGM will be calculated within
#foi = os.path.join(workspace, 'examples.shp')

# Potential vegetation layer
pvurl = 'https://giscog.blob.core.windows.net/publicparquet/Yazoo_PNV.parquet'
pv = gpd.GeoDataFrame(pd.read_parquet(pvurl))
pv['geometry'] = pv['geometry'].apply(wkb.loads)
pv = pv.set_geometry('geometry')
pv = pv.set_crs(5070)

In [5]:
if 'Name' not in foi.columns:
    if uniqueIDName in foi.columns:
        foi = foi.rename(columns={uniqueIDName:'Name'})    
    foi['Name'] = foi['uid']

In [6]:
foi = foi[['Name', 'uid', 'origacres', 'geometry']]

In [7]:
# Read in NLCD for selecting agriculture and woody wetlands.
cogin = 'https://giscog.blob.core.windows.net/newcontainer/nlcd2019_cog.tif'
cog = rasterio.open(cogin)
aoicrs = aoi.to_crs(cog.crs)
nlcd, nlcd_transform =rasterio.mask.mask(cog, aoicrs.dissolve().geometry, crop=True)
ag = nlcd.copy()
ag[ag>82]=0
ag[ag<81]=0
ag[ag==81]=1
ag[ag==82]=1
shapes = rasterio.features.shapes(ag[0], transform=nlcd_transform, mask=ag[0] == 1)
ag = gpd.GeoDataFrame({'geometry': [shape(geom) for geom, value in shapes]})
ag = ag.set_geometry('geometry')
ag = ag.set_crs(5070)

# Our classification
cogwoody = 'https://giscog.blob.core.windows.net/publiccogs/ForestClassification_2023-09-01.tif'
cog = rasterio.open(cogwoody)
aoicrs = aoi.to_crs(cog.crs)
nlcd, nlcd_transform =rasterio.mask.mask(cog, aoicrs.dissolve().geometry, crop=True)
woody = nlcd.copy()
woody[woody>0]=2
woody[woody==0]=1
woody[woody==2]=0

# NLCD woody
#woody = nlcd.copy()
#woody[(woody>90) & (woody <90)]=0
#woody[woody==90]=1
shapes = rasterio.features.shapes(woody[0], transform=nlcd_transform, mask=woody[0] == 1)
woody = gpd.GeoDataFrame({'geometry': [shape(geom) for geom, value in shapes]})
woody = woody.set_geometry('geometry')
woody = woody.set_crs(5070)

In [8]:
# Clip woody to the aoi then remove woody from foi
# Calculates acres after clipping to ag
clippedwoody = gpd.clip(woody, aoi)
try:
    foi = gpd.clip(foi, ag)
except:
    foi['geometry'] = foi.buffer(0)
    foi = gpd.clip(foi, ag)
foi['uid'] = foi.index.map(hash)
foi['acres'] = foi.area * ac_

In [9]:
def getvconnect(gdf):
    # Assume you have an existing GeoDataFrame with MultiLineString geometries
    # Example: Loading a GeoDataFrame from a shapefile or other data source
    # gdf = gpd.read_file('path/to/shapefile.shp')

    # Function to create points every 100 meters along a boundary line
    def create_points_along_line(line, distance):
        num_segments = int(line.length // distance)
        points = [line.interpolate(i * distance) for i in range(num_segments + 1)]
        return points

    # Function to create perpendicular lines that project outside the polygon
    def create_perpendicular_line_outside_polygon(point, line, polygon, length):
        prev_point = line.interpolate(line.project(point) - 0.1)  # Slight offset to calculate direction
        dx, dy = point.x - prev_point.x, point.y - prev_point.y
        angle = math.atan2(dy, dx) + math.pi / 2  # Perpendicular angle (outward)

        # Calculate endpoint of the perpendicular line
        x2 = point.x + length * math.cos(angle)
        y2 = point.y + length * math.sin(angle)

        # Create the perpendicular line (initial guess is outward)
        perpendicular_line = LineString([point, Point(x2, y2)])

        # Check if the end of the line is inside the polygon
        if polygon.contains(Point(x2, y2)):
            # If inside, reverse the direction (flip the angle 180 degrees)
            x2 = point.x - length * math.cos(angle)
            y2 = point.y - length * math.sin(angle)
            perpendicular_line = LineString([point, Point(x2, y2)])

        return perpendicular_line

    # Step 1: Initialize list to store all perpendicular lines
    perpendicular_lines = []

    # Step 2: Process each geometry in the GeoDataFrame
    if isinstance(gdf.geometry, Polygon):
        polygons = [gdf.geometry]  # Single polygon
    elif isinstance(gdf.geometry, MultiPolygon):
        polygons = gdf.geometry.geoms  # Access individual Polygons in MultiPolygon

    # Step 3: Process each polygon individually
    for polygon in polygons:
        boundary_line = polygon.exterior  # Get the boundary line as LineString

        # Create points along the boundary line
        points = create_points_along_line(boundary_line, 100)

        # Create perpendicular lines for each point
        for point in points:
            perpendicular_line = create_perpendicular_line_outside_polygon(point, boundary_line, polygon, 500)
            perpendicular_lines.append(perpendicular_line)

    # Step 5: Create a GeoDataFrame to visualize or save the result
    gdflines = gpd.GeoDataFrame(geometry=perpendicular_lines)
    gdflines = gdflines.set_crs(5070)
    #print(gdf.Name, len(gdflines))
    intersecting_lines = gpd.sjoin(gdflines, clippedwoody, how="inner", predicate="intersects")

    # Step 3: Keep only the unique geometries from the lines_gdf that intersect the polygon
    intersecting_lines_unique = intersecting_lines.drop_duplicates(subset=["geometry"])
    #print('intersecting', len(intersecting_lines_unique))
    return((len(intersecting_lines_unique)/len(gdflines))*100)

In [10]:
'''
Reads in the flood files from ASACE and creates polygons from them.

Calculates intersection of potenvial vegetation and foi
Assigns vegtype based on potential vegetation class.
Calculates vegetation acres before intersection with flood
Intersects foi and flood layer.
Calculates flooded acres
'''
storeflood = []
for fld in floodar:
    flood = rasterio.open(fld)
    flood, flood_transform =rasterio.mask.mask(flood, foi.dissolve().geometry, crop=True)
    shapes = rasterio.features.shapes(flood[0], transform=flood_transform, mask=flood[0]==1)
    geoms = []
    values = []

    for geom,value in shapes:
        geoms.append(shape(geom))
        values.append(value)
    flood_vector = gpd.GeoDataFrame({'geometry': geoms, 'value': values})
    flood_vector = flood_vector.set_crs(5070)
    ######### CHANGE ALL TO BACKWATER
    #foipv = gpd.overlay(foi, pv, how='intersection')
    #foipv['veg']=foipv['HGM'].str.split('-', expand=True)[0]
    #foipv['veg'] = 'backwater'
    #foipv = foipv.dissolve(by=['uid', 'veg'])
    foiflood = foi
    '''
    foiflood['vegtype'] = np.select(
        [
            foiflood['HGM'].str.contains('F-'), 
            foiflood['HGM'].str.contains('RB'),
            foiflood['HGM'].str.contains('RO'),
            foiflood['HGM'].str.contains('D-3'),
            foiflood['HGM'].str.contains('D-4'),
            foiflood['HGM'].str.contains('D-1'),
            foiflood['HGM'].str.contains('D-2')
        ], 
        ['flats', 'backwater', 'overbank','isolated', 'isolated', 'connected', 'connected'], 
        default='Unknown'
    )
    '''
    foiflood['vegtype'] = 'backwater'
    #################
    foiflood['vegacres'] = foiflood.area * ac_
    floodarea = gpd.overlay(foiflood, flood_vector, how='intersection', keep_geom_type=False)
    floodarea = floodarea.dissolve(by=['Name', 'vegtype', 'vegacres'], aggfunc={'value':'mean'})
    floodarea['floodacres'] = floodarea.area * ac_
    floodarea = floodarea.reset_index()
    foiflood = foiflood.merge(floodarea[['Name', 'vegtype', 'floodacres']], on=['Name', 'vegtype'], how='left')
    foiflood['floodacres'] = foiflood['floodacres'].fillna(0)
    #foiflood = foiflood.dissolve(by=['Name', 'vegtype', 'vegacres'], aggfunc={'value':'mean'})
    foiflood = foiflood.reset_index()
    foiflood = foiflood[foiflood['vegtype']!='Unknown']
    foiflood = foiflood.rename(columns={'value':'vfreq'})
    if fld == floodar[0]:
        foiflood['vfreq'] = 2
    else:
        foiflood['vfreq'] = 5
    
    storeflood.append(foiflood)

In [11]:
##################################
# vtract, vcore
##################################

In [12]:
def getvconnect(gdf):
    # Assume you have an existing GeoDataFrame with MultiLineString geometries
    # Example: Loading a GeoDataFrame from a shapefile or other data source
    # gdf = gpd.read_file('path/to/shapefile.shp')
    # Function to create points every 100 meters along a boundary line
    def create_points_along_line(line, distance):
        num_segments = int(line.length // distance)
        points = [line.interpolate(i * distance) for i in range(num_segments + 1)]
        return points

    # Function to create perpendicular lines that project outside the polygon
    def create_perpendicular_line_outside_polygon(point, line, polygon, length):
        prev_point = line.interpolate(line.project(point) - 0.1)  # Slight offset to calculate direction
        dx, dy = point.x - prev_point.x, point.y - prev_point.y
        angle = math.atan2(dy, dx) + math.pi / 2  # Perpendicular angle (outward)

        # Calculate endpoint of the perpendicular line
        x2 = point.x + length * math.cos(angle)
        y2 = point.y + length * math.sin(angle)

        # Create the perpendicular line (initial guess is outward)
        perpendicular_line = LineString([point, Point(x2, y2)])

        # Check if the end of the line is inside the polygon
        if polygon.contains(Point(x2, y2)):
            # If inside, reverse the direction (flip the angle 180 degrees)
            x2 = point.x - length * math.cos(angle)
            y2 = point.y - length * math.sin(angle)
            perpendicular_line = LineString([point, Point(x2, y2)])

        return perpendicular_line

    # Step 1: Initialize list to store all perpendicular lines
    perpendicular_lines = []

    # Step 2: Process each geometry in the GeoDataFrame
    if isinstance(gdf.geometry, Polygon):
        polygons = [gdf.geometry]  # Single polygon
    elif isinstance(gdf.geometry, MultiPolygon):
        polygons = gdf.geometry.geoms  # Access individual Polygons in MultiPolygon

    # Step 3: Process each polygon individually
    for polygon in polygons:
        boundary_line = polygon.exterior  # Get the boundary line as LineString

        # Create points along the boundary line
        points = create_points_along_line(boundary_line, 100)

        # Create perpendicular lines for each point
        for point in points:
            perpendicular_line = create_perpendicular_line_outside_polygon(point, boundary_line, polygon, 500)
            perpendicular_lines.append(perpendicular_line)

    # Step 5: Create a GeoDataFrame to visualize or save the result
    gdflines = gpd.GeoDataFrame(geometry=perpendicular_lines)
    gdflines = gdflines.set_crs(5070)
    #print(gdf.Name, len(gdflines))
    intersecting_lines = gpd.sjoin(gdflines, clippedwoody, how="inner", predicate="intersects")

    # Step 3: Keep only the unique geometries from the lines_gdf that intersect the polygon
    intersecting_lines_unique = intersecting_lines.drop_duplicates(subset=["geometry"])
    #print('intersecting', len(intersecting_lines_unique))
    return((len(intersecting_lines_unique)/len(gdflines))*100)

In [13]:
foi['vcore'] = ((foi['geometry'].buffer(-100).area * ac_)/foi['acres'])*100

In [14]:
'''
Calculates vcore, vtract, and vconnect
vcore is ag area with -100m buffer to calculate ag area core.  Divide by ag area to get percent core.
'''
hold = gpd.GeoDataFrame()
for ft in foi.iterfeatures():
    ws = gpd.GeoDataFrame.from_features([ft],crs="EPSG:5070")
    wsbuffer = ws.copy()
    wsbuffer['geometry'] = ws['geometry'].buffer(10)
    selectwoody = gpd.sjoin(clippedwoody[['geometry']],wsbuffer, how='inner',predicate='intersects')
    ws = gpd.overlay(ws, selectwoody, how='union',keep_geom_type=False)
    ws = ws.dissolve()
    hold = pd.concat([hold, ws])
hold['vtract'] = hold.area * ha_
hold = hold.drop(columns=['index_right'])
hold = hold.rename(columns={'Name_1': 'Name', 'uid_1': 'uid', 'acres_1': 'acres'}).drop(columns=['acres_2', 'uid_2'])

In [15]:
outfoi = foi.merge(hold[['uid', 'vtract']], on='uid')
outfoi['vconnect'] = outfoi.apply(getvconnect, axis=1)

In [16]:
'''
Flats - flats
Riverine backwater - backwater
Riverine overbank - overbank
Isolated depressions - isolated
Connected depressions - connected
'''
# if vtract >= 3000: return 1 else return 0.0003*vtract
def calc_vtract(vtract: float, subclass: str = '') -> float:
    if vtract >= 3000:
        return 1
    else:
        return 0.0003*vtract            

# if vcore >= 40: return 1 else return 0.025*vcore
def calc_vcore(vcore: float, subclass: str = '') -> float:
    if vcore >= 40:
        return 1
    else:
        return 0.025*vcore

# if vconnect >= 40 return 1 else return 0.025*vconnect
def calc_vconnect(vconnect: float, subclass: str = '') -> float:
    if vconnect >= 40:
        return 1
    else:
        return 0.025*vconnect    
    
# if vfreq > 2: return 1.3333+(-0.16667*vfreq) elif vfreq <= 2: return 1
def calc_vfreq(vfreq: float, subclass: str) -> float:
    match subclass:
        case 'flats':
            return 0
        case _:
            if vfreq > 2:
                return 1.3333+(-0.16667*vfreq)
            else:
                return 1

# if vpond >= 70 return 2.16667+(-0.01667*vpond) elif vpond >=40 return 1 else return 0.025*vpond
def calc_vpond(vpond: float, subclass: str) -> float:
    match subclass:
        case 'flats':
            if vpond >= 70:
                return 2.16667+(-0.01667*vpond)
            elif vpond >= 40:
                return 1
            else:
                return 0.025*vpond
        case 'backwater':
            if vpond >= 70:
                return 2.16667+(-0.01667*vpond)
            elif vpond >= 40:
                return 1
            else:
                return 0.025*vpond        
        case 'overbank': #IF(C11>=70,0.5,IF(C11>40,1.666667+(-0.01667*C11),IF(C11>=0,1)))
            if vpond >= 70:
                return 0.5
            elif vpond >= 40:
                return 1.666667+(-0.01667*vpond)
            elif vpond >=0:
                return 1
            else:
                return 0
        case _:
            return 0

            
# if vsoil <=100 return 1-(0.01*vsoil) else 0
def calc_vsoil(vsoil: float, subclass: str = '') -> float:
    if vsoil <= 100:
        return 1-(0.01*vsoil)
    else:
        return 0
    
# if vcec <=100 return 1-(0.01*vcec) else return 0
def calc_vcec(vcec: float, subclass: str) -> float:
    match subclass:
        case 'isolated':
            return 0
        case _:
            if vcec <=100:
                return 1-(0.01*vcec) 
            else:
                return 0
        
# if vtba >=30 return 1 else return 0.03333*vtba
def calc_vtba(vtba: float, subclass: str) -> float:
    match subclass:
        case 'flats':
            if vtba >=30:
                return 1
            else:
                return 0.03333*vtba
        case 'backwater':
            if vtba >=20:
                return 1
            else:
                return 0.05*vtba
        case 'overbank':
            if vtba >= 25:
                return 1
            else:
                return 0.04*vtba
        case 'isolated':
            if vtba >=30:
                return 1
            else:
                return 0.03333*vtba
        case 'connected':
            if vtba >=30:
                return 1
            else:
                return 0.03333*vtba            
    
# if vtden >=800 return 0.5 elif vtden > 500 return 2.25+(-0.0025*vtden) elif vtden >=250 return 1 else return 0.004*vtden
def calc_vtden(vtden: float, subclass: str) -> float:
    match subclass:
        case 'flats':    
            if vtden >= 800:
                return 0.5
            elif vtden > 500:
                return 2.25+(-0.0025*vtden)
            elif vtden >=250:
                return 1
            else:
                return 0.004*vtden
        case 'backwater': # IF(C15>=800,0.5,IF(C15>600,2.5+(-0.0025*C15),IF(C15>=250,1,IF(C15>=0,0+(0.004*C15))))
            if vtden >= 800:
                return 0.5
            elif vtden > 600:
                return 2.25+(-0.0025*vtden)
            elif vtden >=250:
                return 1
            else:
                return 0.004*vtden          
        case 'overbank': # IF(C15>=800,0.5,IF(C15>600,2.5+(-0.0025*C15),IF(C15>=300,1,IF(C15>=0,0+(0.003333*C15))))
            if vtden >= 800:
                return 0.5
            elif vtden > 600:
                return 2.25+(-0.0025*vtden)
            elif vtden >=300:
                return 1
            else:
                return 0.003333*vtden       
        case 'isolated': # IF(C15>=900,0.5,IF(C15>800,5+(-0.005*C15),IF(C15>=250,1,IF(C15>=0,0+(0.004*C15))))
            if vtden >= 900:
                return 0.5
            elif vtden > 800:
                return 5+(-0.005*vtden)
            elif vtden >=250:
                return 1
            else:
                return 0.004*vtden        
        case 'connected': # IF(C15>=800,0.5,IF(C15>700,4.5+(-0.005*C15),IF(C15>=250,1,IF(C15>=0,0+(0.004*C15))))
            if vtden >= 800:
                return 0.5
            elif vtden > 700:
                return 4.5+(-0.005*vtden)
            elif vtden >=250:
                return 1
            else:
                return 0.004*vtden          
    
# if vsnag>=100 return 0.5 elif vsnag >= 80 return 3+(-0.025*vsnag) elif vsnag >=50 return 1 else return 0.02*vsnag    
def calc_vsnag(vsnag: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C16>=100,0.5,IF(C16>=80,3+(-0.025*C16),IF(C16>=50,1,IF(C16>=0,0+(0.02*C16))))
            if vsnag>=100:
                return 0.5
            elif vsnag >= 80:
                return 3+(-0.025*vsnag)
            elif vsnag >=50:
                return 1
            else:
                return 0.02*vsnag
        case 'backwater': # IF(C16>=100,0.5,IF(C16>=60,1.75+(-0.0125*C16),IF(C16>=25,1,IF(C16>=0,0+(0.04*C16))))
            if vsnag>=100:
                return 0.5
            elif vsnag >= 60:
                return 1.75+(-0.0125*vsnag)
            elif vsnag >=25:
                return 1
            else:
                return 0.04*vsnag
        case 'overbank': # IF(C16>=100,0.5,IF(C16>=50,1.5+(-0.01*C16),IF(C16>=15,1,IF(C16>=0,0.5+(0.033333*C16))))
            if vsnag>=100:
                return 0.5
            elif vsnag >= 50:
                return 1.5+(-0.01*vsnag)
            elif vsnag >=15:
                return 1
            else:
                return 0.5+(0.033333*vsnag)
        case 'isolated': # IF(C16>=100,0.5,IF(C16>=70,2.166667+(-0.01667*C16),IF(C16>=30,1,IF(C16>=0,0+(0.03333*C16))))
            if vsnag>=100:
                return 0.5
            elif vsnag >= 70:
                return 2.166667+(-0.01667*vsnag)
            elif vsnag >=30:
                return 1
            else:
                return 0.033333*vsnag
        case 'connected': # IF(C16>=120,0.5,IF(C16>=60,1.5+(-0.00833*C16),IF(C16>=20,1,IF(C16>=0,0+(0.05*C16))))
            if vsnag>=120:
                return 0.5
            elif vsnag >= 60:
                return 1.5+(-0.00833*vsnag)
            elif vsnag >=20:
                return 1
            else:
                return 0.05*vsnag               
            
# if vtcomp >=0 return 0.01*vtcomp else 0
def calc_vtcomp(vtcomp: float, subclass: str) -> float:
    if vtcomp >= 0:
        return 0.01*vtcomp
    else:
        return 0

# if vcomp >= 0 return 0.01*vcomp else return 0
def calc_vcomp(vcomp: float, subclass: str) -> float:
    if vcomp >= 0:
        return 0.01*vcomp
    else:
        return 0

# if vwd >= 130 return 0.5 elif vwd > 90 return 2.125+(-0.0125*vwd) elif vwd >30 return 1 else return 0.033333*vwd
def calc_vwd(vwd: float, subclass: str) -> float:
    match subclass:
        case 'flats':    
            if vwd >= 130:
                return 0.5
            elif vwd > 90:
                return 2.125+(-0.0125*vwd)
            elif vwd >30:
                return 1
            else:
                return 0.033333*vwd
        case 'backwater': # IF(C19>=100,0.5,IF(C19>80,3+(-0.025*C19),IF(C19>=25,1,IF(C19>=0,0+(0.04*C19))))
            if vwd >= 100:
                return 0.5
            elif vwd > 80:
                return 3+(-0.025*vwd)
            elif vwd >25:
                return 1
            else:
                return 0.04*vwd
        case 'overbank': # IF(C19>=100,0.5,IF(C19>80,3+(-0.025*C19),IF(C19>=25,1,IF(C19>=0,0+(0.04*C19))))
            if vwd >= 100:
                return 0.5
            elif vwd > 80:
                return 3+(-0.025*vwd)
            elif vwd >25:
                return 1
            else:
                return 0.04*vwd
        case _:
            return 0

# if vlog >= 100 return 0.5 elif vlog>75, return 2.5+(-0.02*vlog) elif vlog >= 25 return 1 else return 0.04*vlog
def calc_vlog(vlog: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C20>=100,0.5,IF(C20>=75,2.5+(-0.02*C20),IF(C20>=25,1,IF(C20>=0,0+(0.04*C20))))
            if vlog >= 100:
                return 0.5
            elif vlog > 75:
                return 2.5+(-0.02*vlog)
            elif vlog > 25:
                return 1
            else:
                return 0.04*vlog
        case 'backwater': # IF(C20>=100,0.5,IF(C20>=75,2.5+(-0.02*C20),IF(C20>=25,1,IF(C20>=0,0+(0.04*C20))))
            if vlog >= 100:
                return 0.5
            elif vlog > 75:
                return 2.5+(-0.02*vlog)
            elif vlog > 25:
                return 1
            else:
                return 0.04*vlog
        case 'overbank': # IF(C20>=100,0.5,IF(C20>=75,2.5+(-0.02*C20),IF(C20>=25,1,IF(C20>=0,0+(0.04*C20))))
            if vlog >= 100:
                return 0.5
            elif vlog > 75:
                return 2.5+(-0.02*vlog)
            elif vlog > 25:
                return 1
            else:
                return 0.04*vlog
        case _:
            return 0

# if vssd >= 6000 return 0.5 elif vssd >=4000 return 2+(-0.00025*vssd)  else if vssd >= 1500 return 1 else return 0.000667*vssd
def calc_vssd(vssd: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C21>=6000,0.5,IF(C21>=4000,2+(-0.00025*C21),IF(C21>=1500,1,IF(C21>=0,0+(0.000667*C21))))
            if vssd >= 6000:
                return 0.5
            elif vssd >= 4000:
                return 2+(-0.00025*vssd)
            elif vssd >= 1500:
                return 1
            else:
                return 0.000667*vssd
        case 'backwater': # IF(C21>=7500,0.5,IF(C21>=4000,1.571429+(-0.00014*C21),IF(C21>=1500,1,IF(C21>=0,0+(0.000667*C21))))
            if vssd >= 7500:
                return 0.5
            elif vssd >= 4000:
                return 1.571429+(-0.00014*vssd)
            elif vssd >= 1500:
                return 1
            else:
                return 0.000667*vssd
        case 'overbank': # IF(C21>=6000,0.5,IF(C21>=2500,1.357143+(-0.00014*C21),IF(C21>=500,1,IF(C21>=0,0+(0.002*C21))))
            if vssd >= 6000:
                return 0.5
            elif vssd >= 2500:
                return 1.357143+(-0.00014*vssd)
            elif vssd >= 500:
                return 1
            else:
                return 0.002*vssd
        case 'isolated': # IF(C21>=8000,0.5,IF(C21>=4000,1.5+(-0.00013*C21),IF(C21>=1000,1,IF(C21>=0,0+(0.001*C21))))
            if vssd >= 8000:
                return 0.5
            elif vssd >= 4000:
                return 1.5+(-0.00013*vssd)
            elif vssd >= 1000:
                return 1
            else:
                return 0.001*vssd
        case 'connected': # IF(C21>=7000,0.5,IF(C21>=5000,2.25+(-0.00025*C21),IF(C21>=2000,1,IF(C21>=0,0+(0.0005*C21))))
            if vssd >= 7000:
                return 0.5
            elif vssd >= 5000:
                return 2.25+(-0.00025*vssd)
            elif vssd >= 2000:
                return 1
            else:
                return 0.0005*vssd                                
    
# vgvc >80 return 0.05 elif vgvc>=50 return 1.83333+(-0.01667*vgvc) elif vgvc>=10 return 1 else return 0.1*vgvc
def calc_vgvc(vgvc: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C22>=80,0.5,IF(C22>=50,1.83333+(-0.01667*C22),IF(C22>=10,1,IF(C22>=0,0+(0.1*C22))))
            if vgvc >= 80:
                return 0.5
            elif vgvc >= 50:
                return 1.83333+(-0.01667*vgvc)
            elif vgvc >= 10:
                return 1
            else:
                return 0.1*vgvc
        case 'backwater': # IF(C22>=80,0.5,IF(C22>=40,1.5+(-0.0125*C22),IF(C22>=10,1,IF(C22>=0,0+(0.1*C22))))
            if vgvc >= 80:
                return 0.5
            elif vgvc >= 40:
                return 1.5+(-0.0125*vgvc)
            elif vgvc >= 10:
                return 1
            else:
                return 0.1*vgvc
        case 'overbank': # IF(C22>=80,0.5,IF(C22>=40,1.5+(-0.0125*C22),IF(C22>=10,1,IF(C22>=0,0+(0.1*C22))))
            if vgvc >= 80:
                return 0.5
            elif vgvc >= 40:
                return 1.5+(-0.0125*vgvc)
            elif vgvc >= 10:
                return 1
            else:
                return 0.1*vgvc
        case 'isolated': # IF(C22>=70,0.5,IF(C22>=25,1.27778+(-0.01111*C22),IF(C22>=0,1)))
            if vgvc >= 70:
                return 0.5
            elif vgvc >= 25:
                return 1.27778+(-0.01111*vgvc)
            else:
                return 1
        case 'connected': # IF(C22>=70,0.5,IF(C22>=25,1.27778+(-0.01111*C22),IF(C22>=0,1))
            if vgvc >= 70:
                return 0.5
            elif vgvc >= 25:
                return 1.27778+(-0.01111*vgvc)
            else:
                return 1                             

# if vohor >= 2 return 1 elif vohor >= 0 return 0.5+(0.25*vohor)
def calc_vohor(vohor: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C23>=2,1,IF(C23>=0,0.5+(0.25*C23))
            if vohor >= 2:
                return 1
            elif vohor >= 0:
                return 0.5+(0.25*vohor)
            else:
                return 0
        case 'backwater':# IF(C23>=2,1,IF(C23>=0,0.5+(0.25*C23))                        
            if vohor >= 2:
                return 1
            elif vohor >= 0:
                return 0.5+(0.25*vohor)
            else:
                return 0
        case 'overbank':# IF(C23>=1,1,IF(C23>=0,0.5+(0.5*C23))                      
            if vohor >= 2:
                return 1
            elif vohor >= 0:
                return 0.5+(0.5*vohor)
            else:
                return 0
        case _:
            return 0
                            
# if vahor >= 8 return 0.5 elif vahor >= 5 return 1.833333+(-0.16667*vahor) elif vahor >= 2 return 1 else return 0.5+(0.25*vahor)
def calc_vahor(vahor: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C24>=8,0.5,IF(C24>=5,1.833333+(-0.16667*C24),IF(C24>=2,1,IF(C24>=0,0.5+(0.25*C24))))                           
            if vahor >= 8:
                return 0.5
            elif vahor >= 5:
                return 1.833333+(-0.16667*vahor)
            elif vahor >= 2:
                return 1
            else:
                return 0.5+(0.25*vahor)
        case 'backwater': # IF(C24>=8,0.5,IF(C24>=5,1.833333+(-0.16667*C24),IF(C24>=2,1,IF(C24>=0,0.5+(0.25*C24))))                       
            if vahor >= 8:
                return 0.5
            elif vahor >= 5:
                return 1.833333+(-0.16667*vahor)
            elif vahor >= 2:
                return 1
            else:
                return 0.5+(0.25*vahor)
        case 'overbank': # IF(C24>=7,0.5,IF(C24>=4,1.666667+(-0.1667*C24),IF(C24>=2,1,IF(C24>=0,0.5+(0.25*C24))))
            if vahor >= 7:
                return 0.5
            elif vahor >= 4:
                return 1.666667+(-0.1667*vahor)
            elif vahor >= 2:
                return 1
            else:
                return 0.5+(0.25*vahor)
        case _:
            return 0

In [17]:
def calcfci(WAAacreage, vtract, vcore, vconnect, vfreq, rnd=0):
    years = [0,5,10,20,35,50]

    targetyearfci = {
        0:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vdur': 0.5,
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 50,
        'vcec' : 50,
        'vtba' : 0,
        'vtden' : 0,
        'vsnag' : 0,
        'vtcomp' : 0,
        'vcomp' : 0,
        'vwd' : 0,
        'vlog' : 0,
        'vssd' : 0,
        'vgvc' : 0,
        'vohor' : 5,
        'vahor' : 5},
        5:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vdur': 0.5,
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 50,
        'vcec' : 50,
        'vtba' : 0,
        'vtden' : 0,
        'vsnag' : 0,
        'vtcomp' : 89,
        'vcomp' : 89,
        'vwd' : 0,
        'vlog' : 0,
        'vssd' : 538,
        'vgvc' : 65,
        'vohor' : 0.46,
        'vahor' : 1},
        10:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vdur': 0.5,            
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 50,
        'vcec' : 50,
        'vtba' : 3,
        'vtden' : 147,
        'vsnag' : 1,
        'vtcomp' : 93,
        'vcomp' : 93,
        'vwd' : 6,
        'vlog' : 5,
        'vssd' : 966,
        'vgvc' : 65,
        'vohor' : 0.38,
        'vahor' : 1},
        20:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vdur': 0.5,
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 0,
        'vcec' : 0,
        'vtba' : 10,
        'vtden' : 344,
        'vsnag' : 1,
        'vtcomp' : 87,
        'vcomp' : 87,
        'vwd' : 27,
        'vlog' : 17,
        'vssd' : 727,
        'vgvc' : 51,
        'vohor' : 0.56,
        'vahor' : 1},
        35:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vdur': 0.5,
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 0,
        'vcec' : 0,
        'vtba' : 25,
        'vtden' : 725,
        'vsnag' : 33,
        'vtcomp' : 93,
        'vcomp' : 93,
        'vwd' : 38,
        'vlog' : 29,
        'vssd' : 4000,
        'vgvc' : 43,
        'vohor' : 2,
        'vahor' : 3},
        50:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vdur': 0.5,
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 0,
        'vcec' : 0,
        'vtba' : 30,
        'vtden' : 650,
        'vsnag' : 28,
        'vtcomp' : 93,
        'vcomp' : 93,
        'vwd' : 48,
        'vlog' : 40,
        'vssd' : 2500,
        'vgvc' : 30,
        'vohor' : 2,
        'vahor' : 3}              
        }
    # Create Subindex
    allsubindices={}

    for a in ['flats', 'backwater', 'overbank', 'isolated', 'connected']:
        tracksub = {}
        for yr in range(len(years)):
            year = years[yr]
            subindex = {
            'vtract' : calc_vtract(targetyearfci[year]['vtract'], a),
            'vcore' : calc_vcore(targetyearfci[year]['vcore'], a),
            'vconnect' : calc_vconnect(targetyearfci[year]['vconnect'], a),
            'vdur': 0.5,                
            'vfreq' : calc_vfreq(targetyearfci[year]['vfreq'], a),
            'vpond' : calc_vpond(targetyearfci[year]['vpond'], a),
            'vsoil' : calc_vsoil(targetyearfci[year]['vsoil'], a),
            'vcec' : calc_vcec(targetyearfci[year]['vcec'], a),
            'vtba' : calc_vtba(targetyearfci[year]['vtba'], a),
            'vtden' : calc_vtden(targetyearfci[year]['vtden'], a),
            'vsnag' : calc_vsnag(targetyearfci[year]['vsnag'], a),
            'vtcomp' : calc_vtcomp(targetyearfci[year]['vtcomp'], a),
            'vcomp' : calc_vcomp(targetyearfci[year]['vcomp'], a),
            'vwd' : calc_vwd(targetyearfci[year]['vwd'], a),
            'vlog' : calc_vlog(targetyearfci[year]['vlog'], a),
            'vssd' : calc_vssd(targetyearfci[year]['vssd'], a),
            'vgvc' : calc_vgvc(targetyearfci[year]['vgvc'], a),
            'vohor' : calc_vohor(targetyearfci[year]['vohor'], a),
            'vahor' : calc_vahor(targetyearfci[year]['vahor'], a)
            }
            tracksub[year]=subindex
        allsubindices[a]=tracksub

    # Single calculation
    trackyearaccs = {}
    # Create Subindex
    for k,v in allsubindices.items():
        trackallaccs = {}
        #if k != 'backwater':
        #    continue
        for yr in range(len(years)):
            year = years[yr]
            v = allsubindices[k][year]
            subacc = {}
            match k:
                case 'flats':
                    subacc = {
                        'dpfci' : (v['vpond']+v['vohor'])/2,
                        'cnfci': (((v['vtba'] + v['vssd'] + v['vgvc'])/3)+((v['vohor'] + v['vahor'] + v['vwd'] + v['vsnag'])/4))/2,
                        'mpcfci' : ((((v['vtba'] + v['vtden'])/2)+v['vcomp'])/2)*((v['vsoil'] + v['vpond'])/2)**(1/2),
                        'pfwhfci' : ((v['vpond'])*((v['vtcomp'] + v['vsnag'] + v['vtba'])/3)*((v['vlog'] + v['vohor'])/2)*((v['vtract'] + v['vconnect'] + v['vcore'])/3))**(1/4)
                    }
                case 'backwater':
                    subacc = {
                        'dffci' : v['vfreq']*((v['vlog']+v['vgvc']+v['vssd']+v['vtden'])/4),
                        'dpfci' : (v['vpond']+v['vohor'])/2,
                        'cnfci': (((v['vtba']+v['vssd']+v['vgvc'])/3)+((v['vohor']+v['vahor']+v['vwd']+v['vsnag'])/4))/2,
                        'eocfci' : (((v['vdur']*2)+v['vfreq'])/3)*((((v['vohor']+v['vwd']+v['vsnag'])/3)+((v['vtba']+v['vssd']+v['vgvc'])/3))/2),
                        'recpfci' : (((v['vdur']*2)+v['vfreq'])/3)*v['vpond'], #old# v['vfreq']*((v['vcec']+v['vohor']+v['vahor'])/3),
                        'recbfci' : (((v['vdur']*2)+v['vfreq'])/3)*((((v['vohor']+v['vwd']+v['vsnag'])/3)+((v['vtba']+v['vssd']+v['vgvc'])/3))/2),
                        'mpcfci' : (((((v['vtba']+v['vtden'])/2)+v['vcomp'])/2)*(v['vsoil']+v['vpond'])/2)**(1/2),
                        'pfwhfci' : (((v['vdur']+v['vfreq']+v['vpond'])/3)*((v['vtcomp']+v['vsnag']+v['vtba'])/3)*((v['vlog']+v['vohor'])/2)*((v['vtract']+v['vconnect']+v['vcore'])/3))**(1/4)
                    }
                case 'overbank':
                    subacc = {
                        'dffci' : v['vfreq']*((v['vlog']+v['vgvc']+v['vssd']+v['vtden'])/4),
                        'dpfci' : (v['vpond']+v['vohor'])/2,
                        'cnfci': (((v['vtba']+v['vssd']+v['vgvc'])/3)+((v['vohor']+v['vahor']+v['vwd']+v['vsnag'])/4))/2,
                        'eocfci' : (((v['vdur']*2)+v['vfreq'])/3)*((((v['vohor']+v['vwd']+v['vsnag'])/3)+((v['vtba']+v['vssd']+v['vgvc'])/3))/2),
                        'recpfci' : (((v['vdur']*2)+v['vfreq'])/3)*v['vpond'], #v['vfreq']*((v['vcec']+v['vohor']+v['vahor'])/3),
                        'recbfci' : (((v['vdur']*2)+v['vfreq'])/3)*((((v['vohor']+v['vwd']+v['vsnag'])/3)+((v['vtba']+v['vssd']+v['vgvc'])/3))/2),
                        'mpcfci' : (((((v['vtba']+v['vtden'])/2)+v['vcomp'])/2)*(v['vsoil']+v['vpond'])/2)**(1/2),
                        'pfwhfci' : (((v['vdur']+v['vfreq']+v['vpond'])/3)*((v['vtcomp']+v['vsnag']+v['vtba'])/3)*((v['vlog']+v['vohor'])/2)*((v['vtract']+v['vconnect']+v['vcore'])/3))**(1/4)
                    }
                case 'isolated':
                    subacc = {
                        'cnfci': ((v['vtba']+v['vssd']+v['vsnag'])/3),
                        'mpcfci' : (((((v['vtba']+v['vtden'])/2)+v['vcomp'])/2)*v['vsoil'])**(1/2),
                        'pfwhfci' : (((v['vtcomp']+v['vsnag']+v['vtba'])/3)*((v['vtract']+v['vconnect']+v['vcore'])/3))**(1/2)
                    }
                case 'connected':
                    subacc = {
                        'dffci' : v['vfreq']*((v['vssd']+v['vtden'])/2),
                        'cnfci': (v['vtba']+v['vssd']+v['vsnag'])/3,
                        'eocfci' : (((v['vdur']*2)+v['vfreq'])/3)*((v['vtba']+v['vssd']+v['vsnag'])/3),
                        'recpfci' : (((v['vdur']*2)+v['vfreq'])/3)*v['vpond'],
                        'recbfci' : (((v['vdur']*2)+v['vfreq'])/3)*((((v['vohor']+v['vwd']+v['vsnag'])/3)+((v['vtba']+v['vssd']+v['vgvc'])/3))/2),
                        'mpcfci' : (((((v['vtba']+v['vtden'])/2)+v['vcomp'])/2)*v['vsoil'])**(1/2),
                        'pfwhfci' : (((v['vdur']+v['vfreq'])/2)*((v['vtcomp']+v['vsnag']+v['vtba'])/3)*((v['vtract']+v['vcore']+v['vconnect'])/3))**(1/3)
                    }
            if rnd:
                for ky,vl in subacc.items():
                    subacc[ky] = subacc[ky]
            trackallaccs[year]=subacc
        
        trackyearaccs[k]=trackallaccs

    # Calculate AAFCU for each subclass
    functions = ['dffci', 'cnfci', 'dpfci','eocfci', 'recpfci', 'recbfci', 'mpcfci', 'pfwhfci']
    holdfcu={}
    for k,v in trackyearaccs.items():    
        holdfcutemp={}
        for f in functions:
            fcu = []
            for yr in range(len(years)):
                year = years[yr]
                try:
                    curfci = trackyearaccs[k][year][f]
                except:
                    curfci = 0
                if years[yr] == 0:
                    continue
                else:
                    try:
                        prevyear = years[yr-1]
                        prevfci =  trackyearaccs[k][prevyear][f]
                        newfcu = (year-years[yr-1])*(((prevfci+curfci)/3) + ((prevfci+curfci)/6))
                    except:
                        newfcu = 0
                    fcu.append(newfcu)
            holdfcutemp[f]=sum(fcu)/50
        holdfcu[k]=holdfcutemp
        
    df = pd.DataFrame.from_dict(holdfcu)
    df = df.rename(index={'dffci':'Detain Floodwater', 'cnfci':'Cycle Nutrients', 'dpfci':'Detain Precipitation', 'eocfci':'Export Organic Carbon','recpfci':'Physical Remove Elements and Compounds', 'recbfci':'Biological Remove Elements and Compounds','mpcfci':'Maintain Plant Communites','pfwhfci':'Provide Fish and Wildlife Habitat'})
    #df = df.rename(columns={'flats':'Flats','backwater':'Riverine Backwater','overbank':'Riverine Overbank','isolated':'Isolated Depressions','connected':'Connected Depressions'})
    df.loc["Total AAFCU per acre"] = df.sum()
    df.loc["Total AAFCUs per WAA"] = WAAacreage*df.loc["Total AAFCU per acre"]
    return df, trackyearaccs, allsubindices


In [18]:
holdarr=[]
savearr=[]
trackfci = {}
tracksubs = {}
dt = datetime.datetime.now().strftime('%m_%d_%Y')
forcalc = []
for i in range(len(storeflood)):
    indftract = outfoi
    indffreq = storeflood[i][['Name','floodacres', 'vfreq', 'geometry']]
    indf = indftract.merge(indffreq, on='Name')
    indf['geometry'] = indf['geometry_y']
    hold = pd.DataFrame(columns=['Name', 'index', 'flats', 'backwater', 'overbank', 'isolated', 'connected'])
    savesum = pd.DataFrame(columns=['Name', 'vegtype','Total'])
    for l in range(len(indf)):
        WAAacreage = indf.loc[l]['vegacres']
        vtract = indf.loc[l]['vtract']
        vcore = indf.loc[l]['vcore']
        vconnect = indf.loc[l]['vconnect']
        vfreq = indf.loc[l]['vfreq']
        out, trackfci[i], tracksubs[i] = calcfci(WAAacreage,  vtract, vcore, vconnect, vfreq, 1)
        out = out.reset_index()
        out['Name'] = indf.loc[l]['Name']
        out['floodacres'] = indf.loc[l]['floodacres']
        out['vegacres'] = indf.loc[l]['vegacres']
        out = out[['Name','index',indf.loc[l]['vegtype']]]
        savesum = pd.concat([savesum,pd.DataFrame({'Name':indf.loc[l]['Name'],'vegtype':indf.loc[l]['vegtype'],'Total':out.loc[out['index']=='Total AAFCU per acre',indf.loc[l]['vegtype']]*indf.loc[l]['floodacres']})])
        if indf.loc[l]['Name'] in hold['Name'].values:
            hold.loc[(hold['Name'] == indf.loc[l]['Name']),indf.loc[l]['vegtype']] = out[indf.loc[l]['vegtype']]
        else:
            hold = pd.concat([hold, out])
    hold = hold.rename(columns={'index':'Function','flats':'Flats','backwater':'Riverine Backwater','overbank':'Riverine Overbank','isolated':'Isolated Depressions','connected':'Connected Depressions'})
    hold = hold.fillna(0)
    pth = ['alt2_2year', 'alt2_5year']
    #hold.to_csv(os.path.join(workspace,savefile+'_{0}_{1}.csv'.format(pth[i], dt)))
    indf.drop(columns=['uid','vegtype','geometry_x', 'geometry_y', 'geometry']).groupby(['Name', 'origacres', 'acres', 'vcore', 'vtract', 'vconnect', 'vfreq']).sum().to_csv(os.path.join(workspace,savefile+'_{0}_{1}.csv'.format(pth[i], dt)), float_format='%.1f')
    ingdf = gpd.GeoDataFrame(indf)
    #display(ingdf.drop(columns=['geometry_x', 'geometry_y', 'geometry']).style.set_caption("For input into Excel for alt year "+ str(vfreq)))
    pd.options.display.float_format = "{:,.1f}".format
    #display(indf.drop(columns=['uid','vegtype','geometry_x', 'geometry_y', 'geometry']).groupby(['Name', 'origacres', 'acres', 'vcore', 'vtract', 'vconnect', 'vfreq']).sum().style.set_caption("For input into Excel for alt year "+ str(vfreq)))
    forcalc.append(indf.drop(columns=['uid','vegtype','geometry_x', 'geometry_y', 'geometry']).groupby(['Name', 'origacres', 'acres', 'vcore', 'vtract', 'vconnect', 'vfreq']).sum())
    #ingdf.drop(columns=['geometry_x', 'geometry_y']).to_parquet(os.path.join(workspace,savefile+'_{0}_{1}.parquet'.format(pth[i], dt)))
    #ingdf.drop(columns=['geometry_x', 'geometry_y']).to_file(os.path.join(workspace,savefile+'_{0}_{1}.shp'.format(pth[i], dt)))
    holdarr.append(hold)
    savearr.append(savesum)

In [19]:
pd.options.display.float_format = "{:,.1f}".format
for i in range(len(storeflood)):
    if i == 0:
        print('Flood 2')
    else:
        print('Flood 5')
    #display(savearr[i][['Name','Total']].groupby('Name').agg('sum'))

Flood 2
Flood 5


In [20]:
calcdiff = pd.merge(forcalc[0], forcalc[1], on='Name')
calcdiff['addacres'] = calcdiff['floodacres_y']-calcdiff['floodacres_x']
calcdiff = calcdiff.rename(columns={'floodacres_x':'floodacres_2','floodacres_y':'floodacres_5'})
calcdiff = calcdiff.drop(columns=['vegacres_x', 'vegacres_y'])
#calcdiff

In [21]:
indffreq = storeflood[0][['Name','floodacres', 'vfreq', 'geometry']]
indf = outfoi.merge(indffreq, on='Name')
indf = indf.drop(columns=['uid', 'geometry_x', 'geometry_y', 'vfreq', 'floodacres'])
fcucalc = pd.merge(indf, calcdiff, on='Name')
fcucalc.sort_values(by='Name')

,Name,origacres,acres,vegtype,vegacres,vcore,vtract,vconnect,floodacres_2,floodacres_5,addacres
32,0,238.1,230.8,backwater,230.8,48.9,"137,367.0",75.8,78.5,213.3,134.8
31,1,884.9,863.8,backwater,863.8,58.9,"137,623.5",67.9,722.9,790.6,67.7
30,2,455.1,450.4,backwater,450.4,49.3,"137,451.7",82.0,208.3,231.8,23.5
12,3,301.2,298.9,backwater,298.9,65.8,"137,415.3",66.0,174.3,296.9,122.6
27,4,"1,044.8",957.1,backwater,957.1,53.3,"137,697.9",68.2,815.4,933.6,118.2
29,5,302.1,295.9,backwater,295.9,64.0,"137,396.6",58.0,91.3,258.5,167.2
18,6,74.5,74.5,backwater,74.5,35.4,"137,303.7",70.8,62.4,74.3,11.9
15,7,27.8,21.1,backwater,21.1,0.0,"137,282.8",100.0,15.0,21.0,5.9
13,8,436.9,436.9,backwater,436.9,70.7,"137,451.5",77.2,402.1,436.1,34.0
11,9,88.6,78.6,backwater,78.6,30.6,"137,306.6",100.0,75.9,78.2,2.3


In [22]:
savesum = pd.DataFrame(columns=['Tract','ProjectAC', 'RestorationAC', 'floodacres_2', 'aafcu_2','floodacres_5','aafcu_5'])
tosave={}
for l in range(len(fcucalc)):
    WAAacreage = fcucalc.loc[l]['floodacres_2']
    vtract = fcucalc.loc[l]['vtract']
    vcore = fcucalc.loc[l]['vcore']
    vconnect = fcucalc.loc[l]['vconnect']
    out, trackfci[0], tracksubs[0] = calcfci(WAAacreage,  vtract, vcore, vconnect, 2, 1)
    out = out.reset_index()
    out['Name'] = fcucalc.loc[l]['Name']
    tosave['aafcu_2'] = out.loc[out['index']=='Total AAFCU per acre',fcucalc.loc[l]['vegtype']]*fcucalc.loc[l]['floodacres_2']
    WAAacreage = fcucalc.loc[l]['floodacres_5']
    out, trackfci[0], tracksubs[0] = calcfci(WAAacreage,  vtract, vcore, vconnect, 5, 1)
    out = out.reset_index()
    tosave['aafcu_5'] = out.loc[out['index']=='Total AAFCU per acre',fcucalc.loc[l]['vegtype']]*fcucalc.loc[l]['addacres']
    savesum = pd.concat([savesum,pd.DataFrame({'Tract':fcucalc.loc[l]['Name']+1,'ProjectAC':fcucalc.loc[l]['origacres'],'RestorationAC':fcucalc.loc[l]['vegacres'],'floodacres_2':fcucalc.loc[l]['floodacres_2'],'aafcu_2':tosave['aafcu_2'],'floodacres_5':fcucalc.loc[l]['addacres'],'aafcu_5':tosave['aafcu_5']})])
savesum['Total_aafcu'] = savesum['aafcu_2'] +savesum['aafcu_5']
savesum['aafcubyacre'] = savesum['Total_aafcu']/savesum['RestorationAC']
savesum.loc["Total"] = savesum.sum()
display(savesum.sort_values(by='Tract'))

,Tract,ProjectAC,RestorationAC,floodacres_2,aafcu_2,floodacres_5,aafcu_5,Total_aafcu,aafcubyacre
8,1,238.1,230.8,78.5,427.3,134.8,628.4,"1,055.7",4.6
8,2,884.9,863.8,722.9,"3,934.5",67.7,315.6,"4,250.1",4.9
8,3,455.1,450.4,208.3,"1,133.6",23.5,109.4,"1,243.0",2.8
8,4,301.2,298.9,174.3,948.5,122.6,571.7,"1,520.2",5.1
8,5,"1,044.8",957.1,815.4,"4,437.8",118.2,551.2,"4,989.0",5.2
8,6,302.1,295.9,91.3,497.1,167.2,779.4,"1,276.5",4.3
8,7,74.5,74.5,62.4,339.1,11.9,55.3,394.5,5.3
8,8,27.8,21.1,15.0,80.7,5.9,27.3,108.0,5.1
8,9,436.9,436.9,402.1,"2,188.7",34.0,158.5,"2,347.2",5.4
8,10,88.6,78.6,75.9,412.1,2.3,10.8,422.8,5.4
